In [32]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta

In [33]:
def MACD_wzero(df):

    df['EMA12'] = df.Close.ewm(span=12).mean()
    df['EMA26'] = df.Close.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    df['Histogram'] = df.MACD - df.Signal
    dfc = pd.DataFrame()
    dfv = pd.DataFrame()
    df['Señal_venta_MACD_wzero'] = pd.Series(dtype=int)
    df['Señal_compra_MACD_wzero'] = pd.Series(dtype=int)

    
    for i in range(5):
        maskc = (df.Histogram.shift(i) > 0) & (df.Histogram.shift(i+1) < 0) & (df.MACD < 0) & (df.Signal < 0)
        maskv = (df.Histogram.shift(i) < 0) & (df.Histogram.shift(i+1) > 0) & (df.MACD > 0) & (df.Signal > 0)
        dfc = pd.concat([dfc,maskc],axis=1)
        dfv = pd.concat([dfv,maskv],axis=1)
    df['Señal_compra_MACD_wzero'] = np.where(dfc.sum(axis=1),1,0) 
    df['Señal_venta_MACD_wzero'] = np.where(dfv.sum(axis=1),1,0)

    df.set_index('Tiempo',inplace=True)
    df = df.sort_values(by='Tiempo')
    buy = df[df['Señal_compra_MACD_wzero'] == 1][['Close']]
    sell = df[df['Señal_venta_MACD_wzero'] == 1][['Close']]
    buy = buy.Close.astype(float)
    sell = sell.Close.astype(float)

    return buy, sell

def MACD(df):

    df['EMA12'] = df.Close.ewm(span=12).mean()
    df['EMA26'] = df.Close.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    df['Histogram'] = df.MACD - df.Signal
    dfc = pd.DataFrame()
    dfv = pd.DataFrame()
    df['Señal_venta_MACD'] = pd.Series(dtype=int)
    df['Señal_compra_MACD'] = pd.Series(dtype=int)
    
    for i in range(5):
        maskc = (df.Histogram.shift(i) > 0) & (df.Histogram.shift(i+1) < 0)
        maskv = (df.Histogram.shift(i) < 0) & (df.Histogram.shift(i+1) > 0)
        dfc = pd.concat([dfc,maskc],axis=1)
        dfv = pd.concat([dfv,maskv],axis=1)
    df['Señal_compra_MACD'] = np.where(dfc.sum(axis=1),1,0)
    df['Señal_venta_MACD'] = np.where(dfv.sum(axis=1),1,0)
    
    df.set_index('Tiempo',inplace=True)
    df.sort_values(by='Tiempo')
    buy = df[df['Señal_compra_MACD'] == 1][['Close']]
    sell = df[df['Señal_venta_MACD'] == 1][['Close']]
    buy = buy.Close.astype(float)
    sell = sell.Close.astype(float)

    return buy, sell


def buy_sell_prices(buy,sell):
    sellprices = pd.Series(dtype=float)
    buyprices = pd.Series(dtype=float)

    while True:
        if sell.index[0] < buy.index[0]:
            sell = sell.drop(sell.index[0])
        elif buy.index[-1] > sell.index[-1]:
            buy = buy.drop(buy.index[-1])

        if buy.index[0] < sell.index[0] and buy.index[-1] < sell.index[-1]:
            break

    p = 0
    while True:

        if len(buyprices) == 0:
            buyprices = pd.concat([buyprices,buy[0:1]])
            sellprices = pd.concat([sellprices,sell[0:1]])
            j = 0
            continue

        if len(buy[buy.index > buyprices.index[j]]) == p:
                    break

        if len(buyprices) > 0:
            p = 0
            x = len(buyprices)
            while True:    
                if x != len(buyprices):
                    x = len(buyprices)
                    p = 0

                if buy[buy.index > buyprices.index[j]].index[p] > sellprices.index[j]:
                    buyprices = pd.concat([buyprices,buy[buy.index > buyprices.index[j]][p:p+1]])
                    k = 0
                    while True:
                        if buyprices.index[j+1] < sell[sell.index > sellprices.index[j]].index[k]:
                            sellprices = pd.concat([sellprices,sell[sell.index > sellprices.index[j]][k:k+1]])
                            j += 1
                            break
                        k += 1
                p += 1

                if len(buy[buy.index > buyprices.index[j]]) == p:
                    break

    return buyprices, sellprices

def profits_(buyprices,sellprices):
    profits = []

    for i in range(len(buyprices)):
        profit = buyprices[i] - sellprices[i]
        profits.append(profit*0.005)
        net_profit = sum(profits)
    # print('Las ganancias de esta estrategia son de: ', net_profit)
    return net_profit, profits

def logs(net_profit,profits,buyprices,sellprices,kline_interval,days_ago):
    trades = pd.DataFrame([[buyprices,sellprices,profits,]],columns=['buyprices','sellprices','profits'])
    session =  pd.DataFrame([[net_profit,kline_interval,days_ago]],columns=['Ganancia/Perdida','kline_interval','días atras'])
    print(session)

    return session, trades

In [20]:
lista = ['1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1D','3D','1s']
lista2 = []
for i in reversed(lista):
    lista2.append(i)

print(lista2)

['1s', '3D', '1D', '12h', '8h', '6h', '4h', '2h', '1h', '30m', '15m', '5m', '3m', '1m']


In [35]:
# ['1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1D','3D','1w']
# ['1w', '3D', '1D', '12h', '8h', '6h', '4h', '2h', '1h', '30m', '15m', '5m', '3m', '1m']
for j in ['1w', '3D', '1D', '12h', '8h', '6h', '4h', '2h', '1h', '30m', '15m', '5m', '3m', '1m']:

    if j in ['2h', '1h', '30m', '15m', '5m', '3m', '1m']:
        for i in range(5,366):
            interval = j
            days_ago = i
            pair = 'BTCTUSD'
            path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
            df = pd.read_csv(path)[['Tiempo','Close']] 
            df.set_index('Tiempo')
            df.index = pd.to_datetime(df.index)
            df = df.sort_values(by='Tiempo')
            days = df.index[-1] - timedelta(days=days_ago)
            df = df[df.index >= days]

            buy, sell = MACD(df)
            buyprices, sellprices = buy_sell_prices(buy,sell)
            net_profit, profits = profits_(buyprices,sellprices)
            session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)
            session_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/session_by{interval}'
            trade_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/trades_by{interval}'
            if os.path.exists(session_path):
                session.to_csv(session_path,
                               mode='a',
                               header=False,
                               index=False
                            )
            else:
                session.to_csv(session_path,
                               header=True,
                               index=False
                            )
                
            if i == 365:
                trades.to_csv(trade_path,
                              header=True,
                              index=False
                            )

    if j in ['12h', '8h', '6h', '4h']:
        for i in range(25,366):
            interval = j
            days_ago = i
            pair = 'BTCTUSD'
            path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
            df = pd.read_csv(path)[['Tiempo','Close']] 
            df.set_index('Tiempo')
            df.index = pd.to_datetime(df.index)
            df = df.sort_values(by='Tiempo')
            days = df.index[-1] - timedelta(days=days_ago)
            df = df[df.index >= days]

            buy, sell = MACD(df)
            buyprices, sellprices = buy_sell_prices(buy,sell)
            net_profit, profits = profits_(buyprices,sellprices)
            session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)
            session_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/session_by{interval}'
            trade_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/trades_by{interval}'
            if os.path.exists(session_path):
                session.to_csv(session_path,
                               mode='a',
                               header=False,
                               index=False
                            )
            else:
                session.to_csv(session_path,
                               header=True,
                               index=False
                            )
                
            if i == 365:
                trades.to_csv(trade_path,
                              header=True,
                              index=False
                            )

    if j == '1D':
        for i in range(50,366):
            interval = j
            days_ago = i
            pair = 'BTCTUSD'
            path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
            df = pd.read_csv(path)[['Tiempo','Close']] 
            df.set_index('Tiempo')
            df.index = pd.to_datetime(df.index)
            df = df.sort_values(by='Tiempo')
            days = df.index[-1] - timedelta(days=days_ago)
            df = df[df.index >= days]

            buy, sell = MACD(df)
            buyprices, sellprices = buy_sell_prices(buy,sell)
            net_profit, profits = profits_(buyprices,sellprices)
            session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)
            session_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/session_by{interval}'
            trade_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/trades_by{interval}'
            if os.path.exists(session_path):
                session.to_csv(session_path,
                               mode='a',
                               header=False,
                               index=False
                            )
            else:
                session.to_csv(session_path,
                               header=True,
                               index=False
                            )
                
            if i == 365:
                trades.to_csv(trade_path,
                              header=True,
                              index=False
                            )

    if j == '3D':
        for i in range(125,366):
            interval = j
            days_ago = i
            pair = 'BTCTUSD'
            path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
            df = pd.read_csv(path)[['Tiempo','Close']] 
            df.set_index('Tiempo')
            df.index = pd.to_datetime(df.index)
            df = df.sort_values(by='Tiempo')
            days = df.index[-1] - timedelta(days=days_ago)
            df = df[df.index >= days]

            buy, sell = MACD(df)
            buyprices, sellprices = buy_sell_prices(buy,sell)
            net_profit, profits = profits_(buyprices,sellprices)
            session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)
            session_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/session_by{interval}'
            trade_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/trades_by{interval}'
            if os.path.exists(session_path):
                session.to_csv(session_path,
                               mode='a',
                               header=False,
                               index=False
                            )
            else:
                session.to_csv(session_path,
                               header=True,
                               index=False
                            )
                
            if i == 365:
                trades.to_csv(trade_path,
                              header=True,
                              index=False
                            )

    if j == '1w':
        for i in range(275,551):
            interval = j
            days_ago = i
            pair = 'BTCTUSD'
            path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
            df = pd.read_csv(path)[['Tiempo','Close']] 
            df.set_index('Tiempo')
            df.index = pd.to_datetime(df.index)
            df = df.sort_values(by='Tiempo')
            days = df.index[-1] - timedelta(days=days_ago)
            df = df[df.index >= days]

            buy, sell = MACD(df)
            buyprices, sellprices = buy_sell_prices(buy,sell)
            net_profit, profits = profits_(buyprices,sellprices)
            session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)
            session_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/session_by{interval}'
            trade_path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/backtesting/logs/MACD/trades_by{interval}'
            if os.path.exists(session_path):
                session.to_csv(session_path,
                               mode='a',
                               header=False,
                               index=False
                            )
            else:
                session.to_csv(session_path,
                               header=True,
                               index=False
                            )
                
            if i == 550:
                trades.to_csv(trade_path,
                              header=True,
                              index=False
                            )

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
for j in ['4h','6h','8h','12h','1D']:
    for i in range(30,365):
        interval = j
        days_ago = i
        pair = 'BTCTUSD'
        path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
        df = pd.read_csv(path)[['Tiempo','Close']] 
        df.set_index('Tiempo')
        df.index = pd.to_datetime(df.index)
        df = df.sort_values(by='Tiempo')
        days = df.index[-1] - timedelta(days=days_ago)
        df = df[df.index >= days]

        buy, sell = MACD(df)
        buyprices, sellprices = buy_sell_prices(buy,sell)
        net_profit, profits = profits_(buyprices,sellprices)
        session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)

In [21]:
interval = '1h'
days_ago = 28
pair = 'BTCTUSD'
path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
df = pd.read_csv(path)[['Tiempo','Close']] 
df.set_index('Tiempo')
df.index = pd.to_datetime(df.index)
df = df.sort_values(by='Tiempo')
days = df.index[-1] - timedelta(days=days_ago)
df = df[df.index >= days]

buy, sell = MACD(df)
buyprices, sellprices = buy_sell_prices(buy,sell)
net_profit, profits = profits_(buyprices,sellprices)
session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)

   Ganancia/Perdida kline_interval  días atras
0         -167.5617             1h          28


In [28]:
interval = '1h'
days_ago = 28
pair = 'BTCTUSD'
path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
df = pd.read_csv(path)[['Tiempo','Close']] 
df.set_index('Tiempo')
df.index = pd.to_datetime(df.index)
df = df.sort_values(by='Tiempo')
days = df.index[-1] - timedelta(days=days_ago)
df = df[df.index >= days]

buy, sell = MACD_wzero(df)
buyprices, sellprices = buy_sell_prices(buy,sell)
net_profit, profits = profits_(buyprices,sellprices)
session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)

   Ganancia/Perdida kline_interval  días atras
0         -73.05715             1h          28


In [22]:
interval = '4h'
days_ago = 28
pair = 'BTCTUSD'
path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
df = pd.read_csv(path)[['Tiempo','Close']] 
df.set_index('Tiempo')
df.index = pd.to_datetime(df.index)
df = df.sort_values(by='Tiempo')
days = df.index[-1] - timedelta(days=days_ago)
df = df[df.index >= days]

buy, sell = MACD(df)
buyprices, sellprices = buy_sell_prices(buy,sell)
net_profit, profits = profits_(buyprices,sellprices)
session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)

   Ganancia/Perdida kline_interval  días atras
0           10.9046             4h          28


In [29]:
interval = '4h'
days_ago = 28
pair = 'BTCTUSD'
path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
df = pd.read_csv(path)[['Tiempo','Close']] 
df.set_index('Tiempo')
df.index = pd.to_datetime(df.index)
df = df.sort_values(by='Tiempo')
days = df.index[-1] - timedelta(days=days_ago)
df = df[df.index >= days]

buy, sell = MACD_wzero(df)
buyprices, sellprices = buy_sell_prices(buy,sell)
net_profit, profits = profits_(buyprices,sellprices)
session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)

   Ganancia/Perdida kline_interval  días atras
0         110.07475             4h          28


In [30]:
interval = '1d'
days_ago = 28
pair = 'BTCTUSD'
path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
df = pd.read_csv(path)[['Tiempo','Close']] 
df.set_index('Tiempo')
df.index = pd.to_datetime(df.index)
df = df.sort_values(by='Tiempo')
days = df.index[-1] - timedelta(days=days_ago)
df = df[df.index >= days]

buy, sell = MACD(df)
buyprices, sellprices = buy_sell_prices(buy,sell)
net_profit, profits = profits_(buyprices,sellprices)
session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)

   Ganancia/Perdida kline_interval  días atras
0        -107.09055             1d          28


In [31]:
interval = '1d'
days_ago = 28
pair = 'BTCTUSD'
path = f'C:/Users/tinma/OneDrive/Escritorio/tradingbot/csvs/historical/by{interval}/{pair}'
df = pd.read_csv(path)[['Tiempo','Close']] 
df.set_index('Tiempo')
df.index = pd.to_datetime(df.index)
df = df.sort_values(by='Tiempo')
days = df.index[-1] - timedelta(days=days_ago)
df = df[df.index >= days]

buy, sell = MACD_wzero(df)
buyprices, sellprices = buy_sell_prices(buy,sell)
net_profit, profits = profits_(buyprices,sellprices)
session, trades = logs(net_profit,profits,buyprices,sellprices,interval,days_ago)

   Ganancia/Perdida kline_interval  días atras
0           15.4515             1d          28


In [10]:
datetime.utcnow() - timedelta(28)

datetime.datetime(2023, 6, 11, 21, 6, 55, 910377)

In [ ]:
dfx = pd.DataFrame()
